In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableLambda,
)
from dotenv import load_dotenv

In [44]:
load_dotenv()

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Barack_Obama")

doc = loader.load()
doc[0].page_content

'\n\n\n\nBarack Obama - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life and career\n\n\n\n\nToggle Early life and career subsection\n\n\n\n\n\n1.1\nEducation\n\n\n\n\n\n\n\n\n1.2\nFamily and personal life\n\n\n\n\n\n\n\n\n1.3\nReligious views\n\n\n\n\n\n\n\n\n\n\n2\nLegal career\n\n\

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=40)
chunks = splitter.split_documents(doc)

In [46]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

parser = StrOutputParser()

prompt = PromptTemplate(
    template="""
            Answer the question using ONLY the provided context.
            If the answer is not in the context, say "I don't know."

        Context:
        {context}

        Question:
        {question}
        """,
    input_variables=["context", "question"],
)

In [56]:
par_chain = RunnableParallel(
    {
        "context": retriever
        | RunnableLambda(lambda docs: "\n\n".join(doc.page_content for doc in docs)),
        "question": RunnablePassthrough(),
    }
)

fin_chian = par_chain | prompt | llm | parser

fin_chian.invoke("Education of obama")

'Obama attended Occidental College after graduating from high school in 1979. He graduated from Columbia University in 1983 with a Bachelor of Arts degree. He earned a Master of Arts in economics from Harvard University.'